In [1]:
import boto3
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
from folium import GeoJson
from io import BytesIO

In [2]:
import sys
sys.path.append("/agrilearn_app/agrilearn/submodules/commons/")

In [3]:
from agrilearn.crop_classification import evalutate_utils
from agrilearn.crop_classification import yaml_utils
from agrilearn.commons.s3 import s3_utils

from agrilearn.crop_classification import evalutate_utils
from agrilearn.crop_classification import yaml_utils
from agrilearn.commons.s3 import s3_utils

2025-03-05 13:18:03.117219: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-05 13:18:03.134142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741180683.153453    9378 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741180683.159351    9378 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 13:18:03.178929: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Resumo da análise

- Existem colunas duplicadas que representam a mesma variável [``start_season``, ``start_of_season``], além de [``end_season``, ``end_of_season``], [``peak_of_season``, ``peak_season``]
- Existem dados duplicados considerando o ``eopatch_location``
- Não existe mais a coluna ``id_talhao``?

### Global Variables

In [5]:
GEOPACKAGE_PATH = ["/agrilearn_app/datasets/arroz_jan_23_24/geopackage/raw/arroz_jan_670.gpkg", 
                   "/agrilearn_app/datasets/algodao_2022_2023_2023_2023/geopackage/raw/algodao_jan_4751.gpkg", 
                   "/agrilearn_app/datasets/base/geopackage/processed/CORN_73080_SOYBEAN_29670_COTTON_1632_RICE_1172.gpkg",
                   "/agrilearn_app/datasets/base/geopackage/processed/SUGAR_CANE_35276_SOYBEAN_29670_CORN_5710_COTTON_1639_RICE_1173.gpkg"]

label_monitoring_class = 'monitoring_class'
label_eopatch_path = "eopath_location" # 'sentinel_eopatch_current' #eopath_location

# 1. Read Datasets

### Data description
- monitoring_class: é o ``conjunto`` que representa a cultura de interesse do cliente ou da amostra de treinamento (SOYBEAN, CORN)
- period: é o ``texto`` que representa a safra agricola (2023/2024, 2024/2025)
- fonte: é o ``texto`` da origem dos dados (mapas temáticos, banco de dados)
- state: é o ``texto`` do estado do polígono
- area: é o ``ponto flutuante`` da área em métros
- micro: é o ``inteiro`` que representa a micro região
- start_season: é o ``timestamp`` que inicia a safra pra determinada cultura (monitoring_class) e safra (period)
- end_season: é o  ``timestamp`` que termina a safra pra determinada cultura (monitoring_class) e safra (period)
- peak_start: é o ``timestamp`` que inicia o período em que pode ocorrer o pico pra determinada cultura (monitoring_class) e safra (period)
- peak_end: é o ``timestamp`` que termina o período em que pode ocorrer o pico pra determinada cultura (monitoring_class) e safra (period) (Obs: temos um calendário pra isso, um arquivo yaml)
- eopath_location: é o ``texto`` que representa o caminho onde o eopatch está salvo (imagens p/ inferência)
- start_of_season: é o ``timestamp`` da data da emergencia da cultura
- peak_of_season: é o ``timestamp`` da data do pico vegetativo da cultura
- end_of_season: é o ``timestamp`` da data da colheita da cultura
- length_of_season: é o ``inteiro`` com a duração do cultivo em dias
- is_valid_metrics:
- sos_valid:
- pos_valid:
- eos_valid:
- los_valid:
- planting_start: é o ``timestamp`` da data do inicio do plantio.
- planting_end: é o ``timestamp`` da data do fim do plantio.
- start_of_cycle: é o ``timestamp`` da data do inio do cultivo segundo o calendário agrícola
- end_of_cycle: é o ``timestamp`` da data do fim do caledário de cultivo segundo calendário agrícola
- length_of_cycle: é o ``inteiro`` que representa o tamanho do ciclo em dias
- is_valid:
- is_valid_POS:
- is_valid_LOS:
- set_type: é o ``conjunto`` que representar a divisão do dado entre treino, validação e teste
- sampled_date: é o ``texto`` com a data do sample em mês e ano
- cultura_2:
- geometry: é a ``geometria`` do polígono 

In [14]:
gdfs = []

# Lê cada Geopackage e adiciona o GeoDataFrame à lista
for path in GEOPACKAGE_PATH:
    gdf = gpd.read_file(path)
    gdfs.append(gdf)

# Concatena todos os GeoDataFrames em um único GeoDataFrame
if gdfs:
    df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    print(f"Dataset final possui {df.shape[0]} linhas")
    df.head()
else:
    print("Nenhum GeoDataFrame válido encontrado.")

/tmp/ipykernel_9378/3463459671.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))


Dataset final possui 184443 linhas


In [7]:
df.head()

,monitoring_class,period,fonte,state,area,micro,start_season,end_season,peak_start,peak_end,...,key_bucket,local_eopatch_path,total_imagens_do_intervalo,total_de_imagem,status_missing_images,days_gap,dates_diff,mean_dates_diff,status_series_missing,monitoring_class_path
0,RICE,2023/2024,Conab - Arroz Irrigado 2022/2023,TO,5.956645e+04,59.0,2023-10-01,2024-06-01,2023-12-01,2024-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RICE,2023/2024,Conab - Arroz Irrigado 2022/2023,TO,5.129706e+04,59.0,2023-10-01,2024-06-01,2023-12-01,2024-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RICE,2023/2024,Conab - Arroz Irrigado 2022/2023,TO,2.276749e+05,59.0,2023-10-01,2024-06-01,2023-12-01,2024-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RICE,2023/2024,Conab - Arroz Irrigado 2022/2023,TO,4.278957e+05,59.0,2023-10-01,2024-06-01,2023-12-01,2024-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RICE,2023/2024,Conab - Arroz Irrigado 2022/2023,TO,1.113927e+06,59.0,2023-10-01,2024-06-01,2023-12-01,2024-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['start_of_season'].sample()

In [9]:
datetime_columns = ['start_season', 
                    'end_season', 
                    'peak_start', 
                    'peak_end',
                    'start_of_season',
                    'peak_of_season',
                    'end_of_season',
                    'planting_start',
                    'planting_end',
                    'start_of_cycle',
                    'end_of_cycle'                  
                   ]

    # Verifica se a coluna existe, se existir formata, senão printa mensagem
for col in datetime_columns:
    if col in df.columns:
        try:
            df[col] = pd.to_datetime(df[col])
        except Exception as e:
            print(f"Erro ao formatar a coluna {col}: {e}")
    else:
        print(f"A coluna '{col}' não existe no DataFrame.")

# Exibe informações das colunas datetime
df[datetime_columns].info()

Erro ao formatar a coluna start_of_season: unconverted data remains when parsing with format "%Y-%m-%d": "T00:00:00.000", at position 242. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.
Erro ao formatar a coluna end_of_season: unconverted data remains when parsing with format "%Y-%m-%d": "T00:00:00.000", at position 198. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.
<class 'pandas.core.frame.DataFram

In [15]:
df[['start_season', 'start_of_season']].sample(10)

,start_season,start_of_season
88974,2020-09-01 00:00:00,2020/12/02 00:00:00
108358,2020-12-01 00:00:00,2021/02/10 00:00:00
47403,2023-01-01 00:00:00,2023-03-11
67427,2023-01-01 00:00:00,2023-04-01
28014,2022-08-01 00:00:00,2022-10-17
117920,2020-10-01,2020-11-24 00:00:00
160661,2020-04-01,NaT
65132,2023-01-01 00:00:00,2023-03-16
45975,2023-01-01 00:00:00,2023-04-13
63465,2023-01-01 00:00:00,2023-01-27


In [16]:
df['monitoring_class'].value_counts()

monitoring_class
CORN          78790
SOYBEAN       59340
SUGAR_CANE    35276
COTTON         8022
RICE           3015
Name: count, dtype: int64

In [17]:
df['state'].value_counts()

state
SP    29078
PR    29073
MG    26586
MT    24678
GO    17216
RS    12768
MS    12085
TO     6604
MA     5466
SC     4526
BA     3884
RO     3826
PA     3632
PI     3521
DF      745
RR      561
AC      116
AP       78
Name: count, dtype: int64

In [18]:
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns")

There are 184443 rows and 55 columns


In [24]:
df['start_season'].dtype

dtype('O')

In [19]:
print(f"Dados de {df['start_season'].min()} a {df['end_season'].max()}")

TypeError: '<=' not supported between instances of 'str' and 'Timestamp'

In [ ]:
list(df.columns)

# 2. Univariate Data Analysis

In [ ]:
#check NAN values
df.isna().sum()

### 2.1 Analysing Target y ``monitoring_class``

In [ ]:
df['monitoring_class'].nunique()

In [ ]:
pd.concat([df['monitoring_class'].value_counts(),
          df['monitoring_class'].value_counts(normalize=True)*100], axis=1)

### 2.2 Analysing dados por ``estado``

In [ ]:
df['state'].nunique()

In [ ]:
pd.concat([df['state'].value_counts(),
          df['state'].value_counts(normalize=True)*100], axis=1)

### 2.3 Analysis ``period``

In [ ]:
df['period'].nunique()

In [ ]:
pd.concat([df['period'].value_counts(),
          df['period'].value_counts(normalize=True)*100], axis=1)

### 2.4 Analysis ``micro``

In [ ]:
df['micro'] = df['micro'].astype(int)

In [ ]:
df['micro'].nunique()

In [ ]:
pd.concat([df['micro'].value_counts(),
          df['micro'].value_counts(normalize=True)*100], axis=1)

### 2.5 Analysis ``sampled_date``

In [ ]:
df['sampled_date'].nunique()

In [ ]:
pd.concat([df['sampled_date'].value_counts(),
          df['sampled_date'].value_counts(normalize=True)*100], axis=1)

### 2.6 Analysis ``set_type``

In [ ]:
pd.concat([df['set_type'].value_counts(),
          df['set_type'].value_counts(normalize=True)*100], axis=1)

### 2.7 Analysis of ``length_of_season (LOS)``

In [ ]:
df['length_of_season'] = df['length_of_season'].astype(int)

In [ ]:
df['length_of_season'].describe()

In [ ]:
pd.concat([df['length_of_season'].value_counts(),
          df['length_of_season'].value_counts(normalize=True)*100], axis=1)

In [ ]:
df[df['length_of_season'] < 100]

### 2.8 Analysis of ``area``

In [ ]:
pd.concat([df['area'].value_counts(),
          df['area'].value_counts(normalize=True)*100], axis=1)

### 2.9 Analysis of ``length_of_cycle (LOC)``

In [ ]:
df['length_of_cycle'] = df['length_of_cycle'].astype(int)

In [ ]:
df['length_of_cycle'].describe()

In [ ]:
pd.concat([df['length_of_cycle'].value_counts(),
          df['length_of_cycle'].value_counts(normalize=True)*100], axis=1)

### 2.10 Analysis of ``cultura_2``

In [ ]:
#df['cultura_2'].unique()

In [ ]:
# pd.concat([df['cultura_2'].value_counts(),
#           df['cultura_2'].value_counts(normalize=True)*100], axis=1)

### 2.11 Analysis of ``fonte``

In [ ]:
pd.concat([df['fonte'].value_counts(),
          df['fonte'].value_counts(normalize=True)*100], axis=1)

### 2.12 Analysis of ``eopath_location``

In [ ]:
df['eopath_location'].nunique()

In [ ]:
df.shape[0]

In [ ]:
df[df['eopath_location'].duplicated()]['eopath_location']

In [ ]:
# df[df['eopath_location'].duplicated()]['eopath_location'].iloc[0]

### 2.13 Check ``start_season`` and ``end_season``

In [ ]:
df[['start_season', 'end_season']].describe()

In [ ]:
df[df['start_season'] == df['end_season']]

In [ ]:
df['start_season'].dt.year.unique()

In [ ]:
df['end_season'].dt.year.unique()

In [ ]:
df['start_of_season'].dt.year.unique()

In [ ]:
df['end_of_season'].dt.year.unique()

### 2.14 Check ``start_season`` and ``end_season``

In [ ]:
df[['peak_start', 'peak_end']].describe()

In [ ]:
df[df['peak_start'] == df['peak_end']]

In [ ]:
df['peak_start'].dt.year.unique()

In [ ]:
df['peak_end'].dt.year.unique()

### 2.15 Check ``planting_start`` and ``planting_end``

In [ ]:
df[['planting_start', 'planting_end']].describe()

In [ ]:
df[df['planting_start'] == df['planting_end']]

In [ ]:
df['planting_start'].dt.year.unique()

In [ ]:
df['planting_end'].dt.year.unique()

# 3. Process and transform data